In [1]:
from __future__ import division
#matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pymysql
import scipy.stats
from sklearn import linear_model
from sklearn import svm
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import os
import random
import warnings
import shutil
warnings.filterwarnings("ignore")

In [2]:
import io
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums=set()
    else:
        pagenums=set(pages)
        
    output=io.BytesIO()
    manager=PDFResourceManager()
    converter=TextConverter(manager, output, codec='utf-8', laparams=LAParams())
    interpreter=PDFPageInterpreter(manager,converter)
    
    infile=file(fname,'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text=output.getvalue()
    output.close
    return text

In [3]:
import unicodedata
import sys

tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P'))
def remove_punctuation(text):
    return text.translate(tbl)

def extract_quality_features(text,dictionary_words,show=1):
    percent_tokens=[]
    len_tokens=[]
    text_processed=[]
    for i in np.arange(len(text)):
        temp=text[i].replace(u"\u2018", "").replace(u"\u2019", "").replace(u"\n"," ")
        tokens=temp.split()
        tokens=[remove_punctuation(word.lower()) for word in tokens]
        punctuations = re.compile(r'[-./?!,--&":;()|0-9]')
        tokens=[punctuations.sub("", word) for word in tokens if punctuations.sub("", word)]
        text_processed.append(" ".join(tokens))
        len_tokens.append(len(tokens))   

    vect=CountVectorizer(vocabulary=dictionary_words)
    X_counts=vect.transform(text_processed)
    X_counts_sum=X_counts.sum(axis=1)
    X_counts_sum = np.squeeze(np.asarray(X_counts_sum))

    percent_tokens=[]
    for i in range(len(text)):
        temp=X_counts_sum[i]/len_tokens[i]
        if np.isfinite(temp):
            percent_tokens.append(temp if temp<1 else 1)
        else:
            percent_tokens.append(0)
    if show:
        print "Do nothing"
    return percent_tokens,len_tokens,text_processed

In [4]:
os.chdir(r'C:\Users\vbask\Documents\Darts_IP\Darts_IP\Dictionaries')
dictionary_words=pickle.load(open("latin_rus_dic.p","rb"))

In [5]:
gen_OCRed_text_path="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\2_b_Poor_OCRed\\Batch_prior"
os.chdir(gen_OCRed_text_path)

gen_OCRed_text_filename=[f for f in os.listdir(gen_OCRed_text_path) if os.path.isfile(f) and os.path.splitext(f)[1]==".pdf"]
print gen_OCRed_text_filename,
print len(gen_OCRed_text_filename)


['darts-190-906-C-de.pdf', 'darts-226-398-A-de.pdf', 'darts-259-916-A-de.pdf', 'darts-510-738-B-de.pdf', 'darts-639-682-A-de.pdf', 'darts-935-617-C-de.pdf'] 6


In [6]:
import time
time1=time.time()
import os

os.chdir(gen_OCRed_text_path)
gen_OCRed_text=[]
erroneous_fnames=[]
non_erroneous_fnames=[]

for fname in gen_OCRed_text_filename:
    try:
        print("Success "+fname)
        gen_OCRed_text.append(convert(fname).decode('utf-8','ignore'))
        non_erroneous_fnames.append(fname)
        

    except:
        print('Some Error skipped with try')
        print fname
        erroneous_fnames.append(fname)

time2=time.time()
print('\n')
print (time2-time1)
print('Non OCR Prediciton through error')
print erroneous_fnames


Success darts-190-906-C-de.pdf
Success darts-226-398-A-de.pdf
Success darts-259-916-A-de.pdf
Success darts-510-738-B-de.pdf
Success darts-639-682-A-de.pdf
Success darts-935-617-C-de.pdf


2.44000005722
Non OCR Prediciton through error
[]


In [7]:
gen_OCRed_text_percent_tokens,gen_OCRed_text_len_tokens,gen_OCRed_text_processed=extract_quality_features(gen_OCRed_text,dictionary_words,0)

In [8]:
X_array_test=pickle.load(open("C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\1_Poor_downloaded\\X_array_test.p","rb"))
test_text_filename=pickle.load(open("C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\1_Poor_downloaded\\test_text_filename.p","rb"))

In [9]:
poor_downloaded_path="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\1_Poor_downloaded\\Batch_prior"
poor_native_path="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\3_a_Poor_Native"
poor_img_path="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\2_a_Poor_img_layer\\Batch_prior"
poor_scanned_path="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\3_b_Poor_Scanned"

In [10]:
gen_OCRed_text_filename=non_erroneous_fnames
import shutil
for root, dirs, files in os.walk(poor_native_path):
    for f in files:
        os.unlink(os.path.join(root, f))
for root, dirs, files in os.walk(poor_scanned_path):
    for f in files:
        os.unlink(os.path.join(root, f))
        

In [11]:
Native_filename=[];Scanned_filename=[]
for index in range(len(gen_OCRed_text_filename)):
    if gen_OCRed_text_len_tokens[index] < 0.5*X_array_test[test_text_filename.index(gen_OCRed_text_filename[index])][1]:
        Native_filename.append(gen_OCRed_text_filename[index])
        shutil.copy(poor_downloaded_path+"\\"+gen_OCRed_text_filename[index],poor_native_path)
    else:
        Scanned_filename.append(gen_OCRed_text_filename[index])
        shutil.copy(poor_img_path+"\\"+gen_OCRed_text_filename[index],poor_scanned_path)


In [12]:
import polyglot
from polyglot.detect import Detector
for index in range(len(Scanned_filename)):
    print(Scanned_filename[index])
    try:
        for language in Detector(gen_OCRed_text[gen_OCRed_text_filename.index(Scanned_filename[index])],quiet=True).languages:
            print(language)
    except:
        print "Do nothing"
    print("\n")


darts-226-398-A-de.pdf
name: German      code: de       confidence:  99.0 read bytes:  1018
name: un          code: un       confidence:   0.0 read bytes:     0
name: un          code: un       confidence:   0.0 read bytes:     0


darts-259-916-A-de.pdf
name: German      code: de       confidence:  88.0 read bytes:   977
name: English     code: en       confidence:  11.0 read bytes:   440
name: un          code: un       confidence:   0.0 read bytes:     0


darts-935-617-C-de.pdf
name: German      code: de       confidence:  97.0 read bytes:   919
name: un          code: un       confidence:   0.0 read bytes:     0
name: un          code: un       confidence:   0.0 read bytes:     0




In [13]:
ocr_languages=["en","de","fr","es","it","ru","nl","pt","sv","tr","th","zh","ja","pl","ko"]

In [14]:
detected_languages=[]
for index in range(len(Scanned_filename)):
    Confidence=[]
    temp_detected_languages=[]
    for language in Detector(gen_OCRed_text[gen_OCRed_text_filename.index(Scanned_filename[index])],quiet=True).languages:
        try:
            print gen_OCRed_text_filename[gen_OCRed_text_filename.index(Scanned_filename[index])]
            if language.code in ocr_languages:
                Confidence.append(language.confidence)
                temp_detected_languages.append(language.code)
            if sum(Confidence)>85:
                break
        except:
            print "Do nothing"
    if list(set(temp_detected_languages[:2])-set(['en','zh','ru']))==[]:
        detected_languages.append([])
    else:
        detected_languages.append(temp_detected_languages[:2])
    



darts-226-398-A-de.pdf
darts-259-916-A-de.pdf
darts-935-617-C-de.pdf


In [15]:
for index in range(len(Scanned_filename)):
    print Scanned_filename[index],detected_languages[index]

darts-226-398-A-de.pdf ['de']
darts-259-916-A-de.pdf ['de']
darts-935-617-C-de.pdf ['de']


In [16]:
combinations=[]
for lang_1 in ocr_languages:
    for lang_2 in ocr_languages:
        if lang_1!=lang_2:
            combinations.append(lang_1+"_"+lang_2)
    ocr_languages=ocr_languages[1:]
ocr_languages=["en","de","fr","es","it","ru","nl","pt","sv","tr","th","zh","ja","pl","ko"]

In [17]:
for folder in combinations+ocr_languages:
    directory="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\4_a_Re_OCRing_input\\"+folder
    for root, dirs, files in os.walk(directory):
        for f in files:
            os.unlink(os.path.join(root, f))
        
for folder in combinations+ocr_languages:
    directory="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\4_a_Re_OCRing_input\\"+folder
    if not os.path.exists(directory):
        os.makedirs(directory)

In [18]:
re_ocr_in_path="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\4_a_Re_OCRing_input"
re_ocr_out_path="C:\\Users\\vbask\\Documents\\Darts_IP\\Darts_IP\\Factories\\German_Factory\\4_b_Re_OCRed"

for root, dirs, files in os.walk(re_ocr_out_path):
    for f in files:
        try:
            os.unlink(os.path.join(root, f))
        except:
            print" "

In [19]:
for index in range(len(Scanned_filename)):
    index_len=len(detected_languages[index])
    if index_len==2:
        if os.path.exists(re_ocr_in_path+"\\"+detected_languages[index][0]+"_"+detected_languages[index][1]):
            shutil.copy(poor_scanned_path+"\\"+Scanned_filename[index],re_ocr_in_path+"\\"+detected_languages[index][0]+"_"+detected_languages[index][1])
        elif os.path.exists(re_ocr_in_path+"\\"+detected_languages[index][1]+"_"+detected_languages[index][0]):
            shutil.copy(poor_scanned_path+"\\"+Scanned_filename[index],re_ocr_in_path+"\\"+detected_languages[index][1]+"_"+detected_languages[index][0])
    if index_len==1:
        if os.path.exists(re_ocr_in_path+"\\"+detected_languages[index][0]):
            shutil.copy(poor_scanned_path+"\\"+Scanned_filename[index],re_ocr_in_path+"\\"+detected_languages[index][0])
    if index_len==0:
        shutil.copy(poor_scanned_path+"\\"+Scanned_filename[index],re_ocr_out_path)